### Callback API
- 모델이 학습 중에 충돌이 발생하거나 네트워크가 끊기면, 모든 훈련 시간이 낭비될 수 있고,  
  과적합을 방지하기 위해 훈련을 중간에 중지해야 할 수도 있다.
- 모델이 학습을 시작하면 학습이 완료될 때까지 아무런 제어를 하지 못하게 되고,  
  신경망 훈련을 완료하는 데에는 몇 시간 또는 며칠이 걸릴 수 있기 때문에 모델을 모니터링하고 제어할 수 있는 기능이 필요하다.
- 훈련 시(fit()) Callback API를 등록시키면 반복 내에서 특정 이벤트 발생마다 등록된 callback이 호출되어 수행된다.

**ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weight_only=False, mode='auto')**
- 특정 조건에 따라서 모델 또는 가중치를 파일로 저장한다.
- filepath: "weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.hdf5"와 같이 모델의 체크포인트를 저장한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- save_best_only: 가장 좋은 성능을 나타내는 모델을 저장할 지에 대한 여부
- save_weights_only: weights만 저장할 지에 대한 여부
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto


**ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_lr=0)**
- 특정 반복동안 성능이 개선되지 않을 때, 학습률을 동적으로 감소시킨다.
- monitor: 모니터링할 성능 지표를 작성한다.
- factor: 학습률을 감소시킬 비율, 새로운 학습률 = 기존 학습률 * factor
- patience: 학습률을 줄이기 전에 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto


**EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')**
- 특정 반복동안 성능이 개선되지 않을 때, 학습을 조기에 중단한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- patience: Early Stopping을 적용하기 전에 monitor할 반복 횟수.
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

In [1]:
from tensorflow.keras.layers import Layer, Input, Flatten, Dense
from tensorflow.keras.models import Model

INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [5]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [6]:
!dir callback_files

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: B6CD-1218

 D:\KDT_0900_dkh\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-27  오후 01:25    <DIR>          .
2024-05-29  오전 08:12    <DIR>          ..
               0개 파일                   0 바이트
               2개 디렉터리  733,247,246,336 바이트 남음


### ModelCheckPoint

In [12]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# mcp_cb = ModelCheckpoint(
#     filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
#     monitor='val_loss',
#     # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
#     save_best_only=False,
#     save_weights_only=True,
#     mode='min'
# )

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=False,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.7430 - loss: 0.7523 - val_acc: 0.8502 - val_loss: 0.4167
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - acc: 0.8524 - loss: 0.4090 - val_acc: 0.8577 - val_loss: 0.3892
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 690us/step - acc: 0.8700 - loss: 0.3561 - val_acc: 0.8709 - val_loss: 0.3514
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 681us/step - acc: 0.8766 - loss: 0.3329 - val_acc: 0.8727 - val_loss: 0.3437
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - acc: 0.8869 - loss: 0.3079 - val_acc: 0.8696 - val_loss: 0.3521
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - acc: 0.8961 - loss: 0.2862 - val_acc: 0.8749 - val_loss: 0.3385
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 727us/step - acc: 0.8930 - loss: 0.2889 - val_acc: 0.8703 - val_loss: 0.3410
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - acc: 0.9008 - loss: 0.2677 - val_acc: 0.8832 - val_loss: 0.3296
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s

In [8]:
!dir callback_files

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: B6CD-1218

 D:\KDT_0900_dkh\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-29  오전 08:13    <DIR>          .
2024-05-29  오전 08:12    <DIR>          ..
2024-05-29  오전 08:13           739,600 weights.001-0.4016-0.8085.weights.h5
2024-05-29  오전 08:13           739,600 weights.002-0.3736-0.8608.weights.h5
2024-05-29  오전 08:13           739,600 weights.003-0.3669-0.8731.weights.h5
2024-05-29  오전 08:13           739,600 weights.004-0.3357-0.8817.weights.h5
2024-05-29  오전 08:13           739,600 weights.005-0.3419-0.8871.weights.h5
2024-05-29  오전 08:13           739,600 weights.006-0.3258-0.8912.weights.h5
2024-05-29  오전 08:13           739,600 weights.007-0.3230-0.8955.weights.h5
2024-05-29  오전 08:13           739,600 weights.008-0.3355-0.8995.weights.h5
2024-05-29  오전 08:13           739,600 weights.009-0.3161-0.9033.weights.h5
2024-05-29  오전 08:13           739,600 weights.010-0.3123-0.9075.weights.h5
2024-05-29  오전 08:13           739,600 weights.01

In [9]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - acc: 0.8864 - loss: 0.3598


[0.362560898065567, 0.885699987411499]

In [11]:
model = create_model()
model.load_weights('./callback_files/weights.020-0.3222-0.9302.weights.h5')

model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - acc: 0.8864 - loss: 0.3598


[0.362560898065567, 0.885699987411499]

In [13]:
from tensorflow.keras.models import load_model

model = load_model('./callback_files/model.020-0.3500-0.9309.model.keras')
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - acc: 0.8766 - loss: 0.3975


[0.38855838775634766, 0.8791999816894531]

### ReduceLROnPlateau

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[rlr_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - acc: 0.7381 - loss: 0.7544 - val_acc: 0.8390 - val_loss: 0.4400 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 679us/step - acc: 0.8528 - loss: 0.4173 - val_acc: 0.8628 - val_loss: 0.3811 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - acc: 0.8681 - loss: 0.3652 - val_acc: 0.8689 - val_loss: 0.3533 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 654us/step - acc: 0.8769 - loss: 0.3401 - val_acc: 0.8777 - val_loss: 0.3372 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 685us/step - acc: 0.8820 - loss: 0.3224 - val_acc: 0.8784 - val_loss: 0.3320 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 718us/step - acc: 0.8907 - loss: 0.2986 - val_acc: 0.8790 - val_loss: 0.3265 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step - acc: 0.8949 - loss: 0.2845 - val_acc: 0.8798 - val_loss: 0.3228 - learning_rate:

### EarlyStopping

In [15]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 910us/step - acc: 0.7348 - loss: 0.7706 - val_acc: 0.8421 - val_loss: 0.4211
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - acc: 0.8567 - loss: 0.4039 - val_acc: 0.8618 - val_loss: 0.3747
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - acc: 0.8697 - loss: 0.3598 - val_acc: 0.8706 - val_loss: 0.3547
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 711us/step - acc: 0.8774 - loss: 0.3383 - val_acc: 0.8729 - val_loss: 0.3440
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 675us/step - acc: 0.8842 - loss: 0.3126 - val_acc: 0.8766 - val_loss: 0.3360
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 660us/step - acc: 0.8926 - loss: 0.2921 - val_acc: 0.8808 - val_loss: 0.3204
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 662us/step - acc: 0.8927 - loss: 0.2856 - val_acc: 0.8753 - val_loss: 0.3376
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step - acc: 0.8984 - loss: 0.2707 - val_acc: 0.8784 - val_loss: 0.3359
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 

In [16]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 954us/step - acc: 0.7454 - loss: 0.7464 - val_acc: 0.8481 - val_loss: 0.4122 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 729us/step - acc: 0.8585 - loss: 0.3966 - val_acc: 0.8653 - val_loss: 0.3720 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step - acc: 0.8704 - loss: 0.3545 - val_acc: 0.8713 - val_loss: 0.3499 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 727us/step - acc: 0.8800 - loss: 0.3278 - val_acc: 0.8748 - val_loss: 0.3424 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - acc: 0.8865 - loss: 0.3090 - val_acc: 0.8624 - val_loss: 0.3662 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 717us/step - acc: 0.8895 - loss: 0.2972 - val_acc: 0.8752 - val_loss: 0.3473 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - acc: 0.9054 - loss: 0.2563 - val_acc: 0.8860 - val_loss: 0.3072 - learning_rate: